# Class Explanation (クラスに関する説明)

## phase2

### Kalman Filter (kalman.py)

In [1]:
'''Kalman_Filter(observation = None, initial_mean = None, initial_covariance = None, transition_matrices = None, observation_matrices = None, transition_covariance = None, observation_covariance = None, transition_noise_matrices = None, transition_offsets = None, observation_offsets = None, em_vars=['transition_covariance', 'observation_covariance', 'initial_mean', 'initial_covariance'], em_dics = {}, n_dim_sys = None, n_dim_obs = None, dtype = np.float32)'''

"Kalman_Filter(observation = None, initial_mean = None, initial_covariance = None, transition_matrices = None, observation_matrices = None, transition_covariance = None, observation_covariance = None, transition_noise_matrices = None, transition_offsets = None, observation_offsets = None, em_vars=['transition_covariance', 'observation_covariance', 'initial_mean', 'initial_covariance'], em_dics = {}, n_dim_sys = None, n_dim_obs = None, dtype = np.float32)"

状態方程式，観測方程式は以下で与えられている．ただし，オフセット$\mathbf{b}_t,\mathbf{d}_t$，状態ノイズ変換行列$G_t$は抜いても良い．

$$
\begin{align*}
&\mathbf{x}_{t+1}=F_t\mathbf{x}_t+\mathbf{b}_t+G_t\mathbf{v}_t,\ \mathbf{v}_t\sim N(0,Q_t)\\
&\mathbf{y}_t=H_t\mathbf{x}_t+\mathbf{d}_t+\mathbf{w}_t,\ \mathbf{w}_t\sim N(0,R_t)
\end{align*}
$$

必要な変数

|変数名  |対応文字  |デフォルト値 |説明
|:--|:-:|:-:|:--|
|observation  |$\{\mathbf{y}_t\}$ |  |観測変量
|initial_mean  |$E[\mathbf{x}_0]$ |np.zeros |初期状態の平均
|initial_covariance |$V[\mathbf{x}_0]$ |np.eye |初期状態の共分散
|transition_matrices |$\{F_t\}$ |np.eye |状態遷移行列
|observation_matrices |$\{H_t\}$ |np.eye |観測行列
|transition_covariance |$\{Q_t\}$ |np.eye |状態ノイズ共分散行列
|observation_covariance |$\{R_t\}$ |np.eye |観測ノイズ共分散行列
|transition_noise_matrices |$\{G_t\}$ |np.eye |状態ノイズ変換行列
|transition_offsets |$\mathbf{b}_t$ |np.zeros |状態遷移オフセット
|observation_offsets |$\mathbf{d}_t$ |np.zeros |観測オフセット
|em_vars | |$Q,R,E[\mathbf{x}_0],V[\mathbf{x}_0]$ |EMアルゴリズムで最適化したい変数名を与えるリスト型変数
|em_dics | |{} |EMアルゴリズムでパラメータを最適化する際に行列・ベクトルの特定の値を固定したい際に用いる辞書型変数
|n_dim_sys | | |状態変数の次元
|n_dim_obs| | |観測変数の次元
|dtype| |np.float32 |内部で計算する numpy-array のタイプ

格納されている関数

|関数名 |説明
|:--|:--
|filter |一気先予測・フィルターを行う関数
|smooth |RTS平滑化を行う関数
|get_predicted_value(dim=None) |第dim次元の一気先予測値(背景値)を得る関数
|get_filtered_value(dim=None) |第dim次元のフィルター値(解析値)を得る関数
|get_smoothed_value(dim=None) |第dim次元のRTS平滑化値を得る関数
|em(n_iter=10) |EMアルゴリズムを用いてパラメータ(em_vars)をn_iter回イテレーションを回し最適化する関数

em_vars について
- EMアルゴリズムで最適化したい変数をリストで渡す．
    - 例えば，em_vars = ['transition_covariance', 'initial_mean'] と与える
- 最適化可能対象は次のとおりである．
    - 'transition_matrices'
    - 'observation_matrices'
    - 'transition_covariance'
    - 'observation_covariance'
    - 'initial_mean'
    - 'initial_covariance'
    - 'transition_offsets'
    - 'observation_offsets'

em_dics について
- EMアルゴリズム適用時に，行列内，ベクトル内の特定の要素を最適化したい時に用いる．
    - 例えば，状態遷移行列の対角成分だけを固定したいような場合が考えられる．
        - 状態遷移行列がpxp行列の場合，em_dics = {'transition_matrices' : [list(range(p)), list(range(p))]} とすれば良い．
- 辞書の key として，固定したい対象の変数の名前，value として固定したい要素番号を入れて用いる．
    - 2x2状態遷移行列の対角成分を固定したい場合
        - em_dics = {'transition_matrices' : [[0, 1], [0, 1]]}
        - [0,0], [1,1] ではなく，二つのリストを用いて要素の場所を指定していることに注意されたい．
    - 2x2状態遷移行列の非対角成分[0,1]を固定したい場合
        - em_dics = {'transition_matrices' : [[0], [1]]}
    - 5次元状態オフセットベクトルの第0,3成分を固定したい場合
        - em_dics = {'transition_offsets' : [0, 3]}
        - ベクトルに関しては直観的に分かりやすい仕様．
    - pxp状態遷移行列，kxk観測行列の対角成分を固定したい場合
        - em_dics = {'transition_covariance' : [list(range(p)), list(range(p))],
            'observation_covariance' : [list(range(k)), list(range(k))]}
        - 複数のパラメータで固定する成分がある場合は，それぞれkeyを作れば良い
- 使用例は，phase2/180313_Kalman_Filter_for_class_make.ipynb を参照にして欲しい．
    - クラス作りながらメモ書きしたり実験したりする用のため，コードとしてはまとまってないですが．

今後，追加したい機能

|機能 |説明
|:--|:--
|状態方程式の拡張 |$x_{t+1}=F_tx_t+\Gamma_tu_t+G_tv_t$に状態方程式を拡張したい．隠れ状態$x_t$に関する遷移を考えているが，既知変数$u_t$を含めた形への拡張．
|EM Algorithmの時変性 |現状は時不変な$F,H,Q,R$の最適化を行なっているが，時変な$F_t,H_t,Q_t,R_t$への拡張．
|memory saving |RTS smoothing 用に確保しなければいけないメモリをどうするか．

### Ensemble Kalman Filter (ensemble.py)

In [2]:
'''Ensemble_Kalman_Filter(self, observation = None, transition_functions = None, observation_matrices = None, initial_mean = None, transition_noise = None, observation_covariance = None, n_particle = 100, n_dim_sys = None, n_dim_obs = None, dtype = np.float32, seed = 10)'''

'Ensemble_Kalman_Filter(self, observation = None, transition_functions = None, observation_matrices = None, initial_mean = None, transition_noise = None, observation_covariance = None, n_particle = 100, n_dim_sys = None, n_dim_obs = None, dtype = np.float32, seed = 10)'

状態方程式，観測方程式は次のように与えられている．

$$
\begin{align*}
&\mathbf{x}_{t+1}=\mathbf{f}_t(\mathbf{x}_t)+\mathbf{v}_t,\ \mathbf{v}_t\sim p(\mathbf{v}_t)\\
&\mathbf{y}_t=H_t\mathbf{x}_t+\mathbf{w}_t,\ \mathbf{w}_t\sim N(0,R_t)
\end{align*}
$$

必要な変数

|変数名  |対応文字  |デフォルト値 |説明
|:--|:-:|:-:|:--|
|observation  |$\{\mathbf{y}_t\}$ |  |観測変量
|initial_mean  |$E[\mathbf{x}_0]$ |np.zeros |初期状態の平均
|transition_functions |$\{\mathbf{f}_t\}$ |lambda x,v:x+v |状態遷移関数
|observation_matrices |$\{H_t\}$ |np.eye |観測行列
|transition_noise |$\{p(\mathbf{v}_t)\}$ |(np.multivariatenormal, [np.zeros,np.eye]) |パラメトリック状態ノイズ(サイズ変数で指定できる形式)
|observation_covariance |$\{R_t\}$ |np.eye |観測ノイズ共分散行列
|n_particle | |100 |アンサンブルメンバー数
|n_dim_sys | | |状態変数の次元
|n_dim_obs| | |観測変数の次元
|seed | |10 |乱数発生シード
|dtype| |np.float32 |内部で計算する numpy-array の dtype

格納されている関数

|関数名 |説明
|:--|:--
|filter |Evensenの方法で一気先予測・フィルターを行う関数
|smooth(lag=10) |lag間隔で固定ラグ平滑化を行う関数
|get_predicted_value(dim=None) |第dim次元の一気先予測値(背景値)を得る関数
|get_filtered_value(dim=None) |第dim次元のフィルター値(解析値)を得る関数
|get_smoothed_value(dim=None) |第dim次元の平滑化値を得る関数

今後，編集・追加したいこと

|内容 |説明
|:--|:--
|使える乱数の拡張 |現状，numpy で使用可能な乱数を入れる構造になっているが，自己正規化サンプリングや逐次サンプリング等が使えるような拡張の検討
|SVDの速度改善 |SVDの速度をあげられる秘策があれば知りたい．スパースデータならTruncatedSVDとかがあるが．

## Particle Filter (particle.py)

In [1]:
'''Particle_Filter(self, observation = None, initial_mean = None, initial_covariance = None, transition_functions = None, transition_noise = None, likelihood_functions = None, likelihood_function_parameters = None, likelihood_function_is_log_form = True, observation_parameters_time_invariant = True, n_particle = 100, n_dim_sys = None, n_dim_obs = None, dtype = np.float32, seed = 10)'''

'Particle_Filter(self, observation = None, initial_mean = None, initial_covariance = None, transition_functions = None, transition_noise = None, likelihood_functions = None, likelihood_function_parameters = None, likelihood_function_is_log_form = True, observation_parameters_time_invariant = True, n_particle = 100, n_dim_sys = None, n_dim_obs = None, dtype = np.float32, seed = 10)'

状態方程式，観測方程式は次のように与えられている．

$$
\begin{align*}
&\mathbf{x}_{t+1}=\mathbf{f}_t(\mathbf{x}_t)+\mathbf{v}_t,\ \mathbf{v}_t\sim p(\mathbf{v}_t)\\
&\mathbf{y}_t\sim\mathbf{h}_t(\mathbf{y}_t|\mathbf{x}_t)
\end{align*}
$$

必要な変数

|変数名  |対応文字  |デフォルト値 |説明
|:--|:-:|:-:|:--|
|observation  |$\{\mathbf{y}_t\}$ |  |観測変量
|initial_mean  |$E[\mathbf{x}_0]$ |np.zeros |初期状態の平均
|initial_covariance |$E[\mathbf{x}_0\mathbf{x}_0^T]$ |np.eye |初期状態の共分散
|transition_functions |$\{\mathbf{f}_t\}$ |lambda x,v:x+v |状態遷移関数
|transition_noise |$\{p(\mathbf{v}_t)\}$ |(np.multivariatenormal, [np.zeros,np.eye]) |パラメトリック状態ノイズ(サイズ変数で指定できる形式)
|likelihood_functions |$\{\mathbf{h}_t\}$ |Gauss distribution |観測の(対数)尤度関数
|likelihood_function_parameters | |[np.eye(n_dim_obs)] |(対数)尤度関数のパラメータ．Gaussでパラメータだけ変えたい場合は，likelihood_function_parametersだけ変えれば良い．
|likelihood_function_is_log_form | |True |True -> 対数尤度， False -> 尤度
|observation_parameters_time_invariant | |True |True -> 時不変性, False -> 時変性
|n_particle | |100 |アンサンブルメンバー数
|n_dim_sys | | |状態変数の次元
|n_dim_obs| | |観測変数の次元
|seed | |10 |乱数発生シード
|dtype| |np.float32 |内部で計算する numpy-array の dtype

格納されている関数

|関数名 |説明
|:--|:--
|filter |Evensenの方法で一気先予測・フィルターを行う関数
|smooth(lag=10) |lag間隔で固定ラグ平滑化を行う関数
|get_predicted_value(dim=None) |第dim次元の一気先予測値(背景値)を得る関数
|get_filtered_value(dim=None) |第dim次元のフィルター値(解析値)を得る関数
|get_smoothed_value(dim=None) |第dim次元の平滑化値を得る関数

使用例(尤度関数について)

In [2]:
def log_norm_likelihood(y, x, H, R, n_particle) :
    '''
    y [n_dim_obs] {numpy-array, float} 
        : observation
        観測 [観測変数軸]
    mean = H @ x [n_particle, n_dim_obs] {numpy-array, float}
        : mean of normal distribution
        各粒子に対する正規分布の平均 [粒子軸，観測変数軸]
    covariance = R [n_dim_obs, n_dim_obs] {numpy-array, float}
        : covariance of normal distribution
        正規分布の共分散 [観測変数軸]
    '''
    Y = np.zeros((len(y), n_particle))
    Y.T[:] = y
    return (- 0.5 * (Y - H @ x).T @ linalg.pinv(R) @ (Y - H @ x)).diagonal()

In [3]:
likelihood_params = [np.eye(3), obs_sigma**2 * np.eye(3), 50]

- 上記の様に対数尤度関数，対応パラメータを定めたとする
    - 観測yの次元 n_dim_obs=3 を想定している
    - n_particle = 50 としてある
- クラス作成時に次の様に代入すれば，$y_t=Hx_t+w_t, w_t\sim N(0,R)$の観測方程式に適用できる
    - likelihood_functions = log_norm_likelihood
    - likelihood_function_parameters = likelihood_params
    - likelihood_function_is_log_form = True
    - observation_parameters_time_invariant = True

今後，編集・追加したいこと

|内容 |説明
|:--|:--
|システムノイズの拡張 |現状，numpy で使用可能な乱数を入れる構造になっているが，自己正規化サンプリングや逐次サンプリング等が使えるような拡張の検討

## Local Ensemble Transform Kalman Filter (letkf.py)

In [1]:
'''Local_Ensemble_Transform_Kalman_Filter(self, observation = None, transition_functions = None, observation_functions = None, initial_mean = None, transition_noise = None, observation_covariance = None, system_adjacency_matrix = None, observation_adjacency_matrix = None, rho = 1, n_particle = 100, n_dim_sys = None, n_dim_obs = None, dtype = np.float32, seed = 10, cpu_number = 'all')'''

"Local_Ensemble_Transform_Kalman_Filter(self, observation = None, transition_functions = None, observation_functions = None, initial_mean = None, transition_noise = None, observation_covariance = None, system_adjacency_matrix = None, observation_adjacency_matrix = None, rho = 1, n_particle = 100, n_dim_sys = None, n_dim_obs = None, dtype = np.float32, seed = 10, cpu_number = 'all')"

状態方程式，観測方程式は次のように与えられている．

$$
\begin{align*}
&\mathbf{x}_{t+1}=\mathbf{f}_t(\mathbf{x}_t)+\mathbf{v}_t,\ \mathbf{v}_t\sim p(\mathbf{v}_t)\\
&\mathbf{y}_t=\mathbf{h}_t(\mathbf{x}_t)+\mathbf{w}_t,\ \mathbf{w}_t\sim N(\mathbf{0},R_t)
\end{align*}
$$

必要な変数

|変数名  |対応文字  |デフォルト値 |説明
|:--|:-:|:-:|:--|
|observation  |$\{\mathbf{y}_t\}$ |  |観測変量
|initial_mean  |$E[\mathbf{x}_0]$ |np.zeros |初期状態の平均
|initial_covariance |$E[\mathbf{x}_0\mathbf{x}_0^T]$ |np.eye |初期状態の共分散
|transition_functions |$\{\mathbf{f}_t\}$ |lambda x,v:x+v |状態遷移関数
|observation_functions |$\{\mathbf{h}_t\}$ |lambda x:x |観測関数
|observation_covariance |$\{R_t\}$ |np.eye |観測共分散
|transition_noise |$\{p(\mathbf{v}_t)\}$ |(np.multivariatenormal, [np.zeros,np.eye]) |パラメトリック状態ノイズ(サイズ変数で指定できる形式)
|system_adjacency_matrix | |np.eye |状態変数の隣接行列（ローカル変換用）
|observation_adjacency_matrix | |np.eye |観測変数の隣接行列（ローカル変数用）
|rho | |1 |multipliative covariance inflating factor
|n_particle | |100 |アンサンブルメンバー数
|n_dim_sys | | |状態変数の次元
|n_dim_obs| | |観測変数の次元
|seed | |10 |乱数発生シード
|dtype| |np.float32 |内部で計算する numpy-array の dtype
|cpu_number| |'all' |並列処理するCPUの数

格納されている関数

|関数名 |説明
|:--|:--
|filter |一気先予測・フィルターを行う関数
|get_predicted_value(dim=None) |第dim次元の一気先予測値(背景値)を得る関数
|get_filtered_value(dim=None) |第dim次元のフィルター値(解析値)を得る関数

今後，編集・追加したいこと

|内容 |説明
|:--|:--
|ローカル変換の追加 |隣接行列でローカル変換を指定しているが，それ以外の方法を模索したい．
|Rの計算の並列処理の模索 |観測共分散を局所空間に移す処理に関しては，並列処理を引き続き模索．